In [1]:
public_r_image = "public.ecr.aws/r4k6a8l9/cookbook-custom-image-01:1"
!docker pull {public_r_image}

1: Pulling from r4k6a8l9/cookbook-custom-image-01
Digest: sha256:331b45592738fafbfcd2c0e9595b86a2eaf24cf807a90642c88aebc53ddff73a
Status: Downloaded newer image for public.ecr.aws/r4k6a8l9/cookbook-custom-image-01:1
public.ecr.aws/r4k6a8l9/cookbook-custom-image-01:1


In [2]:
results = !aws sts get-caller-identity | jq -r ".Account"
account_id = results[0]
account_id

'581320662326'

In [31]:
!aws ecr get-login-password --region us-east-1 | docker login --username AWS --password-stdin {account_id}.dkr.ecr.us-east-1.amazonaws.com

WARNING! Your password will be stored unencrypted in /home/ec2-user/.docker/config.json.
Configure a credential helper to remove this warning. See
https://docs.docker.com/engine/reference/commandline/login/#credentials-store

Login Succeeded


In [29]:
repository_name = "cookbook-custom-image-a"

In [30]:
!aws ecr describe-repositories --repository-names {repository_name} || aws ecr create-repository --repository-name {repository_name} || aws ecr create-repository --repository-name {repository_name}


An error occurred (RepositoryNotFoundException) when calling the DescribeRepositories operation: The repository with name 'cookbook-custom-image-a' does not exist in the registry with id '581320662326'
{
    "repository": {
        "repositoryArn": "arn:aws:ecr:us-east-1:581320662326:repository/cookbook-custom-image-a",
        "registryId": "581320662326",
        "repositoryName": "cookbook-custom-image-a",
        "repositoryUri": "581320662326.dkr.ecr.us-east-1.amazonaws.com/cookbook-custom-image-a",
        "createdAt": 1619872114.0,
        "imageTagMutability": "MUTABLE",
        "imageScanningConfiguration": {
            "scanOnPush": false
        },
        "encryptionConfiguration": {
            "encryptionType": "AES256"
        }
    }
}


In [32]:
results = !aws ecr describe-repositories --repository-names {repository_name} | jq -r ".repositories[].repositoryUri"
target_repository_uri = results[0]
target_repository_uri

'581320662326.dkr.ecr.us-east-1.amazonaws.com/cookbook-custom-image-a'

In [34]:
!docker tag {public_r_image} {target_repository_uri}:1

Make sure that the exection role is access to ECR

In [37]:
!docker push {target_repository_uri}:1

The push refers to repository [581320662326.dkr.ecr.us-east-1.amazonaws.com/cookbook-custom-image-a]

e14b33c1: Preparing 
ae8be976: Preparing 
19e427b9: Preparing 
eabc4e9a: Preparing 
98cf53cf: Preparing 
adb27acb: Preparing 
ce80292c: Preparing 
d81ed546: Preparing 
0372e6e2: Preparing 
34ef6390: Preparing 
55bd8174: Preparing 
d9e0f36f: Preparing 
fec5d051: Preparing 
62c63edc: Preparing 
31e7ba39: Preparing 
55bd8174: Pushed   580.6MB/572.8MB1: digest: sha256:331b45592738fafbfcd2c0e9595b86a2eaf24cf807a90642c88aebc53ddff73a size: 3676


In [70]:
custom_model_file = "https://github.com/PacktPublishing/" + \
    "Amazon-SageMaker-Cookbook/raw/master/" + \
    "Chapter12/files/model.r.custom.tar.gz"

In [39]:
!mkdir -p tmp

In [40]:
!wget -O tmp/model.r.custom.tar.gz {custom_model_file}

--2021-05-01 12:39:46--  https://github.com/PacktPublishing/Amazon-SageMaker-Cookbook/raw/master/Chapter12/files/model.r.custom.tar.gz
Resolving github.com (github.com)... 140.82.114.4
Connecting to github.com (github.com)|140.82.114.4|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/PacktPublishing/Amazon-SageMaker-Cookbook/master/Chapter12/files/model.r.custom.tar.gz [following]
--2021-05-01 12:39:46--  https://raw.githubusercontent.com/PacktPublishing/Amazon-SageMaker-Cookbook/master/Chapter12/files/model.r.custom.tar.gz
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.111.133, 185.199.108.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1741 (1.7K) [application/octet-stream]
Saving to: ‘tmp/model.r.custom.tar.gz’

tmp/model.r.custom. 100%[===================>] 

In [41]:
s3_bucket = "sagemaker-cookbook-bucket"
prefix = "chapter12"

In [42]:
model_data = f"s3://{s3_bucket}/{prefix}/files/model.r.custom.tar.gz"

In [43]:
!aws s3 cp tmp/model.r.custom.tar.gz {model_data}

upload: tmp/model.r.custom.tar.gz to s3://sagemaker-cookbook-bucket/chapter12/files/model.r.custom.tar.gz


In [44]:
import sagemaker
sagemaker_session = sagemaker.Session()
role = sagemaker.get_execution_role()

In [45]:
image = f"{target_repository_uri}:1"
container = { 
    'Image': image,
    'ModelDataUrl': model_data,
    'Mode': 'SingleModel'
}

In [54]:
model_name = 'cookbook-single-custom-model'
endpoint_config_name = 'cookbook-single-custom-model-endpoint-config'
endpoint_name = 'cookbook-single-custom-model-endpoint'

In [55]:
import boto3
sm_client = boto3.client('sagemaker')

response = sm_client.create_model(
    ModelName = model_name,
    ExecutionRoleArn = role,
    Containers = [container])

response

{'ModelArn': 'arn:aws:sagemaker:us-east-1:581320662326:model/cookbook-single-custom-model',
 'ResponseMetadata': {'RequestId': '151ff5a5-d7fd-4993-b14b-24147ec83eea',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': '151ff5a5-d7fd-4993-b14b-24147ec83eea',
   'content-type': 'application/x-amz-json-1.1',
   'content-length': '90',
   'date': 'Sat, 01 May 2021 12:48:59 GMT'},
  'RetryAttempts': 0}}

In [56]:
response = sm_client.create_endpoint_config(
    EndpointConfigName=endpoint_config_name,
    ProductionVariants=[{
        'InstanceType': 'ml.t2.medium',
        'InitialInstanceCount': 1,
        'InitialVariantWeight': 1,
        'ModelName': model_name,
        'VariantName': 'AllTraffic'}])

response

{'EndpointConfigArn': 'arn:aws:sagemaker:us-east-1:581320662326:endpoint-config/cookbook-single-custom-model-endpoint-config',
 'ResponseMetadata': {'RequestId': 'b5d00172-c3b8-4b79-9b0d-23690449a938',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': 'b5d00172-c3b8-4b79-9b0d-23690449a938',
   'content-type': 'application/x-amz-json-1.1',
   'content-length': '125',
   'date': 'Sat, 01 May 2021 12:49:01 GMT'},
  'RetryAttempts': 0}}

In [57]:
response = sm_client.create_endpoint(
    EndpointName=endpoint_name,
    EndpointConfigName=endpoint_config_name)

response

{'EndpointArn': 'arn:aws:sagemaker:us-east-1:581320662326:endpoint/cookbook-single-custom-model-endpoint',
 'ResponseMetadata': {'RequestId': 'eeb9c74d-83b0-4cb2-8d02-e5976f3c33c4',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': 'eeb9c74d-83b0-4cb2-8d02-e5976f3c33c4',
   'content-type': 'application/x-amz-json-1.1',
   'content-length': '105',
   'date': 'Sat, 01 May 2021 12:49:07 GMT'},
  'RetryAttempts': 0}}

In [61]:
response = sm_client.describe_endpoint(
    EndpointName=endpoint_name
)

response['EndpointStatus']

'Creating'

In [62]:
from time import sleep

while response['EndpointStatus'] == "Creating":
    print("Still creating...")
    sleep(60)
    
    response = sm_client.describe_endpoint(
        EndpointName=endpoint_name
    )

Still creating...
Still creating...
Still creating...


In [63]:
response

{'EndpointName': 'cookbook-single-custom-model-endpoint',
 'EndpointArn': 'arn:aws:sagemaker:us-east-1:581320662326:endpoint/cookbook-single-custom-model-endpoint',
 'EndpointConfigName': 'cookbook-single-custom-model-endpoint-config',
 'ProductionVariants': [{'VariantName': 'AllTraffic',
   'DeployedImages': [{'SpecifiedImage': '581320662326.dkr.ecr.us-east-1.amazonaws.com/cookbook-custom-image-a:1',
     'ResolvedImage': '581320662326.dkr.ecr.us-east-1.amazonaws.com/cookbook-custom-image-a@sha256:331b45592738fafbfcd2c0e9595b86a2eaf24cf807a90642c88aebc53ddff73a',
     'ResolutionTime': datetime.datetime(2021, 5, 1, 12, 49, 11, 317000, tzinfo=tzlocal())}],
   'CurrentWeight': 1.0,
   'DesiredWeight': 1.0,
   'CurrentInstanceCount': 1,
   'DesiredInstanceCount': 1}],
 'EndpointStatus': 'InService',
 'CreationTime': datetime.datetime(2021, 5, 1, 12, 49, 8, 140000, tzinfo=tzlocal()),
 'LastModifiedTime': datetime.datetime(2021, 5, 1, 12, 57, 24, 125000, tzinfo=tzlocal()),
 'ResponseMetada

In [64]:
runtime_sm_client = boto3.client('sagemaker-runtime')

In [65]:
body = "42"

In [67]:
response = runtime_sm_client.invoke_endpoint(
    EndpointName=endpoint_name,
    ContentType='text/csv',
    Body=body)

response

{'ResponseMetadata': {'RequestId': '80b0d30e-e66e-4e03-a2e7-4cc348510c34',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': '80b0d30e-e66e-4e03-a2e7-4cc348510c34',
   'x-amzn-invoked-production-variant': 'AllTraffic',
   'date': 'Sat, 01 May 2021 12:59:17 GMT',
   'content-type': 'application/json',
   'content-length': '16'},
  'RetryAttempts': 0},
 'ContentType': 'application/json',
 'InvokedProductionVariant': 'AllTraffic',
 'Body': <botocore.response.StreamingBody at 0x7fa37ea12a20>}

In [68]:
response['Body'].read()

b'1233.18227456514'

In [69]:
response = sm_client.delete_endpoint(
    EndpointName=endpoint_name
)

response

{'ResponseMetadata': {'RequestId': '13ff14e4-143c-4300-9d80-080cb780f28a',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': '13ff14e4-143c-4300-9d80-080cb780f28a',
   'content-type': 'application/x-amz-json-1.1',
   'content-length': '0',
   'date': 'Sat, 01 May 2021 13:21:19 GMT'},
  'RetryAttempts': 0}}